In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import urllib.request
import os
import numpy as np
import pandas as pd
from urllib.parse import quote_plus          
import time
from urllib.request import (urlopen, urlparse, urlunparse, urlretrieve)
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import math
import re
from selenium.webdriver.chrome.service import Service
import os 
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import pdb
import os
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import traceback         
from selenium.webdriver.common.proxy import Proxy, ProxyType
import csv
import requests
import json
import random
import psutil
from datetime import datetime
import pickle
import warnings

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-setuid-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--allow-running-insecure-content')
chrome_options.add_argument("disable-infobars")
#chrome_options.add_argument("headless")
#chrome_options.add_argument("--start-maximized")
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
chrome_options.add_argument(f'user-agent={user_agent}')
os.environ['WDM_LOG_LEVEL'] = '0'
os.environ['WDM_LOG'] = "false"
warnings.filterwarnings("ignore")
chrome_path = 'C:\\Users\\cafel\\chromedriver.exe'

In [3]:
def selenium_scroll_down(driver):
    SCROLL_PAUSE_SEC = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_SEC)
        new_height = driver.execute_script("return document.body.scrollHeight")
  
        if new_height == last_height:
            return 1
        last_height = new_height

def get_driver(chrome_path, chrome_options, url):
    driver = None
    count = 0
    
    while driver is None and count < 10:
        try:
            driver = webdriver.Chrome(chrome_path, options=chrome_options)
        except Exception as e:
            count += 1
            print(f"Error creating driver: {e}")
            clean_up(driver)
            continue
    
    connect = False
    
    while not connect: 
        try:
            driver.get(url)
            driver.implicitly_wait(10)
            connect = True
        except Exception as e:
            print(f"Error connecting to URL: {e}")
            clean_up(driver)
            driver = webdriver.Chrome(chrome_path, options = chrome_options)

    return driver

def clean_up(driver):
    # Define your clean-up logic here
    pass

def reset_driver(driver, chrome_path, chrome_options, url):
    try:
        driver.quit()
        driver = get_driver(chrome_path, chrome_options, url)
    except Exception as e:
        print(f"Error resetting driver: {e}")
        driver = get_driver(chrome_path, chrome_options, url)
    
    return driver

def write_data(write_file, datas):
    for data in datas:
        write_file = write_file.append(data, ignore_index=True)
    
    return write_file

def get_list(chrome_path, chrome_options):
    total = pd.DataFrame()

    for i in range(600, 650):
        base_url = 'https://www.economist.com/finance-and-economics?page={}'.format(i)
        try:
            driver = get_driver(chrome_path, chrome_options, base_url)
        except Exception as e:
            print(f"Error creating driver: {e}")
            continue  # Skip to the next iteration on error
        
        wait = WebDriverWait(driver, 10)
        
        try:
            # Handle iFrame
            iframe = driver.find_element(By.CSS_SELECTOR, "#sp_message_iframe_921614")
            driver.switch_to.frame(iframe)
            driver.find_element(By.CSS_SELECTOR, '#notice > div.message-component.message-row.teg-stackable.teg-footer > div.message-component.message-column.teg-stackable--accept > button').click()
            
            page_exist = True
            datas = []

            while page_exist:
                try:
                    main = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'section-collection.section-collection--finance-and-economics.ds-layout-grid.ds-layout-grid--edged'))).find_element(By.CLASS_NAME, 'layout-section-collection.ds-layout-grid')
                    articles = main.find_elements(By.CLASS_NAME, 'css-e6sfh4.e1mdktgm0')

                    for article in tqdm(articles):
                        data = {}
                        try:
                            url = article.find_element(By.CSS_SELECTOR, 'div > h3 > a').get_attribute('href')
                        except Exception:
                            url = None

                        data['url'] = url
                        datas.append(data)

                    page_exist = False

                except Exception as e:
                    print(f"Error scraping data: {e}")
                    break

        finally:
            driver.close()

        dataframe = pd.DataFrame(datas)
        total = pd.concat([total, dataframe])

    total = total.drop_duplicates(subset=['url'], keep='first')
    total.to_csv('C:/Users/cafel/Desktop/The_Economist/article_url_total_2.csv', encoding='utf-8-sig', index=False)
    print("End")

In [36]:
get_list(chrome_path, chrome_options)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  5.27it/s]


End


In [4]:
url_list = pd.read_csv('C:/Users/cafel/Desktop/The_Economist/article_url_total_1.csv', encoding = 'utf-8-sig')

In [14]:
def get_article(chrome_path, chrome_options, url_list):
    login_url = 'https://myaccount.economist.com/s/login/'
    username = 'cafelate789@gmail.com'
    password = 'tea@250803'
    
    # Set up the Chrome driver
    driver = webdriver.Chrome(chrome_path, options=chrome_options)
    driver.get(login_url)
    time.sleep(5)
    
    # Find the login elements and input credentials
    username_input = driver.find_element(By.XPATH, '//*[@id="input-6"]')
    password_input = driver.find_element(By.XPATH, '//*[@id="input-8"]')
    submit_button = driver.find_element(By.XPATH, '/html/body/div[3]/div[3]/div[1]/div/div/div/div[2]/div/div/c-lwc-login-form/div/lightning-card/article/div[2]/slot/div[2]/div[5]/lightning-button/button')

    username_input.send_keys(username)
    password_input.send_keys(password)
    submit_button.click()

    # Wait for the login process to complete
    WebDriverWait(driver, 10).until(EC.url_changes(login_url))

    total = pd.DataFrame()

    for i in tqdm(range(len(url_list))):
        scrapping_url = url_list['url'][i]
        driver.get(scrapping_url)
        
        try:          
            page_exist = True
            datas = []
            
            try:
                # Handle iFrame
                iframe = driver.find_element(By.CSS_SELECTOR, "#sp_message_iframe_921614")
                driver.switch_to.frame(iframe)
                driver.find_element(By.CSS_SELECTOR, '#notice > div.message-component.message-row.teg-stackable.teg-footer > div.message-component.message-column.teg-stackable--accept > button').click()
            
            except:
                continue
                
            while page_exist:
                try:
                    selenium_scroll_down(driver)
                    main = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'css-9rt3sc.e10tor440'))).find_element(By.CLASS_NAME, 'css-1bfdg3q.e1pkqesp0')
                    paragraphs = main.find_elements(By.CLASS_NAME, 'css-1hno3qs.e190yofl0')
                    try:
                        date_text = main.find_element(By.CSS_SELECTOR, 'div.css-1qjp74c.e8tkvfk0 > div:nth-child(3) > div > div.css-1fiks87.e1rnrkkh0 > div > div > div:nth-child(1) > time').get_attribute('datetime')[:10]
                        date = datetime.strptime(date_text, '%Y-%m-%d')
                        title = main.find_element(By.CSS_SELECTOR, 'div.css-1qjp74c.e8tkvfk0 > div:nth-child(1) > section > h1').text
                    except:
                        date = None
                        title = None
                    
                    data = {'date': date, 'title': title}
                    datas.append(data)

                    page_exist = False

                except Exception as e:
                    print(f"Error scraping data: {e}")
                    break

        except:
            continue

        dataframe = pd.DataFrame(datas)
        total = pd.concat([total, dataframe])

    total.to_csv('C:/Users/cafel/Desktop/The_Economist/article.csv', encoding='utf-8-sig', index=False)
    print("End")

In [15]:
get_article(chrome_path, chrome_options, url_list)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:21<00:00,  1.64s/it]


End


In [10]:
def get_article(chrome_path, chrome_options, url_list):
    login_url = 'https://myaccount.economist.com/s/login'
    username = 'cafelate789@gmail.com'
    password = 'tea@250803'
    
    # 세션 생성
    session = requests.Session()

    # 로그인 요청을 위한 데이터
    login_data = {
        'username': username,
        'password': password
    }
    
    # 로그인 요청
    login_response = session.post(login_url, data = login_data)
    
    if login_response.status_code == 200:
        print('로그인 성공')
    else:
        print('로그인 실패')
        
    time.sleep(5)
        
    total = pd.DataFrame()

    for i in tqdm(range(len(url_list))):
        scrapping_url = url_list['url'][i]
        scraping_response = session.get(scrapping_url)
        
        try:          
            page_exist = True
            datas = []

            while page_exist:
                try:
                    selenium_scroll_down(driver)
                    main = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'css-9rt3sc.e10tor440'))).find_element(By.CLASS_NAME, 'css-1bfdg3q e1pkqesp0')
                    paragraphs = main.find_elements(By.CLASS_NAME, 'css-1hno3qs.e190yofl0')
                    try:
                        date_text = main.find_element(By.CSS_SELECTOR, 'div.css-1qjp74c.e8tkvfk0 > div:nth-child(3) > div > div.css-1fiks87.e1rnrkkh0 > div > div > div:nth-child(1) > time').get_attribute('datetime').text[:10]
                        print(date_text)
                        date = datetime.strptime(date_text, '%Y-%m-%d')
                        title = main.find_element(BY.CSS_SELCTOR, 'div.css-1qjp74c.e8tkvfk0 > div:nth-child(1) > section > h1').text
                        print(title)
                    except:
                        date = None
                        title = None
                    
                    data['date'] = date
                    data['title'] = title
                    datas.append(data)

                    page_exist = False

                except Exception as e:
                    print(f"Error scraping data: {e}")
                    break

        finally:
            session.close()

        dataframe = pd.DataFrame(datas)
        total = pd.concat([total, dataframe])

    total = total.drop_duplicates(subset=['url'], keep='first')
    total.to_csv('C:/Users/cafel/Desktop/The_Economist/article.csv', encoding='utf-8-sig', index=False)
    print("End")